# CAMEL Agent Tracking with AgentOps

This notebook demonstrates how to use AgentOps to track and monitor CAMEL agents. We'll cover:
1. Setting up CAMEL with AgentOps
2. Running a single agent example
3. Creating a multi-agent conversation
4. Analyzing session data and costs

## Setup

First, let's ensure we have all required packages installed and set up our environment.

In [1]:
import sys
import subprocess

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Install required packages if not present
required_packages = ["camel-ai", "agentops"]
for package in required_packages:
    try:
        __import__(package.replace("-", "_"))
    except ImportError:
        print(f"Installing {package}...")
        install_package(package)
        print(f"{package} installed successfully!")

Installing camel-ai...



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


camel-ai installed successfully!


In [2]:
import os
import agentops
from camel.agents import ChatAgent
from camel.messages import BaseMessage
from camel.types import ModelPlatformType, ModelType
from camel.models import ModelFactory
import openai

# Helper function to clean API keys
def clean_api_key(key):
    if key:
        return key.strip().strip('"').strip("'")
    return None

# Get and validate API keys
openai_key = clean_api_key(os.getenv("OPENAI_API_KEY"))
agentops_key = clean_api_key(os.getenv("AGENTOPS_API_KEY"))

if not openai_key:
    raise ValueError("OPENAI_API_KEY environment variable is not set")
if not agentops_key:
    raise ValueError("AGENTOPS_API_KEY environment variable is not set")

# Configure OpenAI client
openai.api_key = openai_key
if openai_key.startswith('sk-proj-'):
    openai.api_base = "https://api.openai.com/v1"  # Ensure using standard API endpoint

2024-12-12 08:56:56,604 - camel - INFO - Camel library logging has been configured.


## Single Agent Example

Let's create a simple example where we use a single CAMEL agent to explain Python concepts while tracking the interaction with AgentOps.

In [3]:
# Initialize AgentOps client
ao_client = agentops.Client()
ao_client.configure(api_key=agentops_key)

# Initialize client and start session
try:
    session = ao_client.initialize()
    if not session:
        session = ao_client.start_session(tags=["camel_example", "single_agent"])

    if not session:
        raise RuntimeError("Failed to create AgentOps session")
except Exception as e:
    print(f"Error initializing AgentOps: {e}")
    raise

# Create a tracked version of ChatAgent
class TrackedChatAgent(ChatAgent):
    def __init__(self, system_message, model, name="agent", session=None):
        super().__init__(system_message=system_message, model=model)
        self.name = name
        self.session = session

    def step(self, user_msg):
        try:
            response = super().step(user_msg)
            # Record the interaction with basic parameters
            if self.session:
                try:
                    self.session.record(
                        agentops.LLMEvent(
                            model=str(self.model_backend.model_type),
                            prompt=user_msg,
                            completion=response.msgs[0].content
                        )
                    )
                except Exception as e:
                    print(f"Failed to record event: {e}")
            return response
        except Exception as e:
            print(f"Error during agent step: {e}")
            raise

# Set up the agent
sys_msg = BaseMessage.make_assistant_message(
    role_name="Python Expert",
    content="You are a helpful Python programming expert who can explain concepts clearly and concisely."
)

# Configure model with GPT-3.5
model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI,
    model_type=ModelType.GPT_3_5_TURBO,
)

# Create the agent with session
camel_agent = TrackedChatAgent(
    system_message=sys_msg,
    model=model,
    name="python_expert",
    session=session
)

# Run the agent
user_msg = "What are Python decorators and how do they work?"
print("User:", user_msg)

response = camel_agent.step(user_msg)
print("\nAssistant:", response.msgs[0].content)

# Get analytics
print("\nSession Analytics:")
analytics = session.get_analytics()
print(f"Total Messages: {session.event_counts['llms']}")
print(f"Total Costs: ${session.token_cost:.4f}\n")

# Print detailed analytics
print("Detailed Analytics:")
for key, value in analytics.items():
    print(f"{key}: {value}")

# End the session
session.end_session(end_state="Success", end_state_reason="Completed CAMEL AI example")

🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=c998bf8f-4145-4cc6-a9d0-50a3c3894aad


🖇 AgentOps: Failed to track an agent with the @track_agent decorator.


User: What are Python decorators and how do they work?


2024-12-12 08:57:01,344 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


2024-12-12 08:57:01,539 - camel.agents.chat_agent - INFO - Model gpt-3.5-turbo, index 0, processed these messages: [{'role': 'system', 'content': 'You are a helpful Python programming expert who can explain concepts clearly and concisely.'}, {'role': 'user', 'content': 'What are Python decorators and how do they work?'}]



Assistant: Python decorators are a powerful and advanced feature that allows you to modify or extend the behavior of functions or methods without changing their actual code. Decorators are implemented using the "@" symbol followed by the decorator function name, placed above the function definition.

Here's a simple example to illustrate how decorators work:

```python
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

@my_decorator
def say_hello():
    print("Hello!")

say_hello()
```

In this example, the `my_decorator` function is a decorator that adds some functionality before and after calling the `say_hello` function. When `say_hello` is called, it actually calls the `wrapper` function defined inside the `my_decorator` function, which then calls the original `say_hello` function.

Decorators are commonly used for 

🖇 AgentOps: Session Stats - Duration: 3.8s | Cost: $0.000362 | LLMs: 2 | Tools: 0 | Actions: 0 | Errors: 0


🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=c998bf8f-4145-4cc6-a9d0-50a3c3894aad


Decimal('0.00036249999999999997501998194593397784046828746795654296875')

## Multi-Agent Conversation

Now let's demonstrate how to track a conversation between two CAMEL agents using AgentOps.

In [4]:
# Create another agent for multi-agent conversation
sys_msg_2 = BaseMessage.make_assistant_message(
    role_name="Student",
    content="You are a curious student who asks follow-up questions about Python concepts."
)

# Create second agent with session
student_agent = TrackedChatAgent(
    system_message=sys_msg_2,
    model=model,
    name="student",
    session=session
)

# Run multi-agent conversation
print("Starting multi-agent conversation...")
student_msg = "Can you explain more about how to create decorators that accept arguments?"
print("\nStudent:", student_msg)

expert_response = camel_agent.step(student_msg)
print("\nPython Expert:", expert_response.msgs[0].content)

student_followup = "Could you provide a practical example of when to use decorators with arguments?"
print("\nStudent:", student_followup)

expert_final = camel_agent.step(student_followup)
print("\nPython Expert:", expert_final.msgs[0].content)

# Get final analytics
print("\nSession Analytics:")
analytics = session.get_analytics()
print(f"Total Messages: {session.event_counts['llms']}")
print(f"Total Costs: ${session.token_cost:.4f}\n")

# Print detailed analytics
print("Detailed Analytics:")
for key, value in analytics.items():
    print(f"{key}: {value}")

# End the session
session.end_session(end_state="Success", end_state_reason="Completed CAMEL AI example")

🖇 AgentOps: Failed to track an agent with the @track_agent decorator.


Starting multi-agent conversation...

Student: Can you explain more about how to create decorators that accept arguments?


2024-12-12 08:57:04,170 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


🖇 AgentOps: Could not record event. Start a session by calling agentops.start_session().


2024-12-12 08:57:04,197 - camel.agents.chat_agent - INFO - Model gpt-3.5-turbo, index 0, processed these messages: [{'role': 'system', 'content': 'You are a helpful Python programming expert who can explain concepts clearly and concisely.'}, {'role': 'user', 'content': 'What are Python decorators and how do they work?'}, {'role': 'assistant', 'content': 'Python decorators are a powerful and advanced feature that allows you to modify or extend the behavior of functions or methods without changing their actual code. Decorators are implemented using the "@" symbol followed by the decorator function name, placed above the function definition.\n\nHere\'s a simple example to illustrate how decorators work:\n\n```python\ndef my_decorator(func):\n    def wrapper():\n        print("Something is happening before the function is called.")\n        func()\n        print("Something is happening after the function is called.")\n    return wrapper\n\n@my_decorator\ndef say_hello():\n    print("Hello!


Python Expert: To create decorators that accept arguments, you need to define a higher-order function that takes the arguments for the decorator and returns the actual decorator function. Here's an example to demonstrate how to create a decorator that accepts arguments:

```python
def repeat(num_times):
    def decorator_repeat(func):
        def wrapper(*args, **kwargs):
            for _ in range(num_times):
                result = func(*args, **kwargs)
            return result
        return wrapper
    return decorator_repeat

@repeat(num_times=3)
def greet(name):
    print(f"Hello, {name}!")

greet("Alice")
```

In this example, the `repeat` function is a higher-order function that takes the `num_times` argument and returns the actual decorator function `decorator_repeat`. The `decorator_repeat` function then defines the `wrapper` function, which will execute the original function `func` multiple times based on the `num_times` argument.

When the `greet` function is decorated w

2024-12-12 08:57:07,507 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


🖇 AgentOps: Could not record event. Start a session by calling agentops.start_session().


2024-12-12 08:57:07,532 - camel.agents.chat_agent - INFO - Model gpt-3.5-turbo, index 0, processed these messages: [{'role': 'system', 'content': 'You are a helpful Python programming expert who can explain concepts clearly and concisely.'}, {'role': 'user', 'content': 'What are Python decorators and how do they work?'}, {'role': 'assistant', 'content': 'Python decorators are a powerful and advanced feature that allows you to modify or extend the behavior of functions or methods without changing their actual code. Decorators are implemented using the "@" symbol followed by the decorator function name, placed above the function definition.\n\nHere\'s a simple example to illustrate how decorators work:\n\n```python\ndef my_decorator(func):\n    def wrapper():\n        print("Something is happening before the function is called.")\n        func()\n        print("Something is happening after the function is called.")\n    return wrapper\n\n@my_decorator\ndef say_hello():\n    print("Hello!


Python Expert: Sure! One practical example of using decorators with arguments is implementing a rate-limiting decorator. This decorator can restrict how often a function can be called within a certain time frame. Here's an example implementation:

```python
import time

def rate_limit(limit_per_second):
    def decorator_rate_limit(func):
        last_invocation = 0

        def wrapper(*args, **kwargs):
            nonlocal last_invocation
            current_time = time.time()
            elapsed_time = current_time - last_invocation

            if elapsed_time < 1 / limit_per_second:
                print(f"Rate limit exceeded. Please wait before calling {func.__name__} again.")
            else:
                last_invocation = current_time
                return func(*args, **kwargs)

        return wrapper
    return decorator_rate_limit

@rate_limit(limit_per_second=2)
def api_call():
    print("API call made.")

# Test the rate-limited function
for _ in range(5):
    api_cal

## Conclusion

In this notebook, we demonstrated how to:
1. Track single-agent CAMEL interactions with AgentOps
2. Monitor multi-agent conversations
3. Collect analytics on message counts and costs
4. Manage different agent roles and personalities

For more information, visit:
- [CAMEL AI Documentation](https://www.camel-ai.org/)
- [AgentOps Dashboard](https://app.agentops.ai/)
- [AgentOps Documentation](https://docs.agentops.ai/)

---
Generated with AgentOps - Observability for AI Agents
